In [10]:
import json
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# color-blind friendly color configuration based on ggplot
color_list = ["#E69F00", "#56B4E9", "#009E73", "#F0E442", "#D55E00", "#0072B2",  "#CC79A7"]

### Make MBTA API request

In [11]:
CLIENT_KEY = '26e129e96c2249cc8478881689ee208c'

# url = 'https://opendata.arcgis.com/datasets/d9966f318b924c0399eebe475f9cdcfd_0.geojson'

fetched = []

offset = 0
page = 2000

total = 2500 # over 10000000 rows

while offset <= total - page:
    url = "https://services1.arcgis.com/ceiitspzDAHrdGO1/arcgis/rest/services/GatedStationEntries/FeatureServer/0/query&f=json&outFields=*&where=1=1&resultOffset={}&resultRecordCount={}".format(offset, page)
    print(url)
    try:
        results = requests.get(url, headers={'x-api-key':'26e129e96c2249cc8478881689ee208c'}).json()
        print(results)

    #     fetched += [feature['attributes'] for feature in results['features']]
        offset += page
    #     print(offset)
    except json.decoder.JSONDecodeError as err:
        print('API error: {}'.format(err))
        offset = total
    
df = pd.DataFrame(data=fetched, columns=['station_name', 'stop_id', 'service_date', 'time_period', 'station_entries', 'ObjectId']) #, delim_whitespace=True)
df.head()

https://services1.arcgis.com/ceiitspzDAHrdGO1/arcgis/rest/services/GatedStationEntries/FeatureServer/0/query&f=json&outFields=*&where=1=1&resultOffset=0&resultRecordCount=2000
API error: Expecting value: line 1 column 1 (char 0)


,station_name,stop_id,service_date,time_period,station_entries,ObjectId


In [12]:
%debug

> <ipython-input-9-b6a317f794ed>(25)<module>()
     23         offset += page
     24     #     print(offset)
---> 25     except JSONDecodeError:
     26         print('API error: {}'.format(JSONDecodeError))
     27         offset = total

ipdb> quit()


In [ ]:
# combine time_period and service_date

df['service_date'] =  pd.to_datetime(df['service_date'], unit='ms')
# time_period is an integer marking the start of the 15-minute time-delta
df['time_period'] = df['time_period']//100 * 60 + df['time_period'] % 100
df['service_timedelta'] = pd.to_timedelta(df['time_period'], unit='min')
df['service_time'] = df['service_date'] + df['service_timedelta']
df.head()

In [ ]:
cleaned_df = df[['service_time', 'station_name', 'station_entries']]
cleaned_df.head()

In [ ]:
# cleaned_df['service_time_int'] = cleaned_df.service_time.astype(np.int64)
cleaned_df.loc[:,'service_time_int'] = cleaned_df.service_time.astype(np.int64)

fig, ax = plt.subplots(figsize=(10,6))
cleaned_df.plot(kind='scatter', x='service_time_int', y='station_entries', ax=ax)
ax.set_xticklabels([datetime.fromtimestamp(ts / 1e9).strftime('%D %H:%M:%S') for ts in ax.get_xticks()])
ax.set_xlabel('Service time')
ax.set_ylabel('Num entries')
plt.show()

### Summary statistics

In [ ]:
cleaned_df.info()

In [ ]:
cleaned_df.dropna(inplace=True)
cleaned_df.shape

In [ ]:
# cleaned_df.drop(['service_time_int'], axis=1, inplace=True)

cleaned_df.set_index('service_time', inplace=True)

# time_indexed_df = cleaned_df.groupby(['service_time'], axis=1).reset_index()
cleaned_df.head()

In [ ]:
cleaned_df.head()

In [ ]:
cleaned_df.shape

In [ ]:
# cleaned_df.loc[:,'service_year'] = cleaned_df['service_time'].dt.strftime('%Y')
cleaned_df.loc[:,'service_month'] = cleaned_df['service_time'].dt.strftime('%B')
cleaned_df.loc[:,'service_weekday'] = cleaned_df['service_time'].dt.strftime('%A')
# cleaned_df.loc[:,'service_month'] = cleaned_df.service_time.month
# cleaned_df.loc[:,'service_weekday'] = cleaned_df.service_time.weekday_name

In [ ]:
cleaned_df.head()

In [ ]:
# monthly totals per station

monthly_df = cleaned_df.loc[:, ['station_entries', 'station_name', 'service_weekday']]
grouped_monthly_df = monthly_df.groupby(['service_weekday', 'station_name']).sum().reset_index()
# grouped_monthly_df = monthly_df.groupby('service_weekday').agg({'station_entries' : 'sum',
#                      'station_name' : lambda x: x.iloc[0]})
grouped_monthly_df.head()

ordered_days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
grouped_monthly_df['service_weekday'] = pd.Categorical(grouped_monthly_df['service_weekday'], categories=ordered_days, ordered=True)
grouped_monthly_df = grouped_monthly_df.sort_values('service_weekday')

ax = sns.boxplot(data=grouped_monthly_df, x='service_weekday', y='station_entries', color="lightsteelblue")
ax.set_xlabel('Service weekday')
ax.set_ylabel('Station entries')
ax.set_title('Station entries by weekday')

# Add transparency to colors
for patch in ax.artists:
 r, g, b, a = patch.get_facecolor()
 patch.set_facecolor((r, g, b, .7))

In [ ]:
grouped_monthly_df.head(10)

In [ ]:
grouped_monthly_df.columns

In [ ]:
# if I reindex to datetime, things solve themselves